In [77]:
from bs4 import BeautifulSoup
import urllib2
import csv
from pymongo import MongoClient
from pprint import pprint
import numpy as np
import urllib
import pandas as pd
from random import randint
import json
from scipy.sparse.linalg import svds

In [78]:
movie = pd.read_csv('moviedata.csv')
movies_data = pd.read_csv('Movies.csv')
user_rating = pd.read_csv('User_rating.csv')

## Input

In [79]:
for i in range(0,5):
    movieid = randint(1,100)
    a = movies_data.loc[movies_data['MovieID'] == movieid,['Name']].iloc[0]['Name']
    rated = input("Enter rating for: " + a)
    user_rating = user_rating.append({'UserID': 1000, 'MovieID': movieid, 'Rated': rated}, ignore_index=True)

Enter rating for: The Marvelous Mrs. Maisel3
Enter rating for: Miss Bala4
Enter rating for: Close2
Enter rating for: Stan & Ollie3
Enter rating for: A Simple Favor1


In [80]:
movie_data = pd.merge(movies_data,user_rating, on='MovieID')
movie_user_rating = movie_data.pivot_table(index= 'Name',columns = 'UserID',values='Rated',aggfunc= 'first')
movie_user_rating.head(5)

UserID,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
Name,,,,,,,,,,,,,,,,,,,,,
A Million Little Things,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN
A Quiet Place,NaN,5.0,NaN,NaN,1.0,NaN,3.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Series of Unfortunate Events,NaN,NaN,NaN,NaN,5.0,NaN,NaN,2.0,NaN,NaN,...,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
A Simple Favor,NaN,NaN,3.0,2.0,NaN,NaN,NaN,2.0,NaN,4.0,...,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
A Star Is Born,NaN,5.0,NaN,NaN,NaN,4.0,NaN,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN


## Item - Item

In [82]:
usersrating= user_rating.loc[user_rating['UserID'] == 1000, ['Rated']] #get the new users rating
max2 = usersrating['Rated'].astype(float).idxmax() #find the index of the maximum rated movie
if user_rating.iloc[max2]['Rated']>3: #See if user has given any of the movie more than 3 stars
    max_movie_id = user_rating.iloc[max2]['MovieID'] #Get the movie ID from the index
    max_movie_name1 = movies_data.loc[movies_data['MovieID'] == max_movie_id, ['Name']] #Get the name of the movie from ID
    max_movie_name = max_movie_name1.iloc[0]['Name']
    movie_data = pd.merge(movies_data,user_rating, on='MovieID') #merge the 2DFs
    user_movie_rating=movie_data.pivot_table(index='UserID',columns='Name',values='Rated',aggfunc='first').astype(float)#utility Matrix
    ratings = user_movie_rating[str(max_movie_name)].astype(float)#Get the ratings of the max rated movie
    movie_like=user_movie_rating.corrwith(ratings,method='pearson')#Find the movies with similar ratings(pearsonSimilarity)
    corr= pd.DataFrame(movie_like, columns=['Correlation']) 
    corr.dropna(inplace=True)
    corr = corr.sort_values('Correlation',ascending=False).head(6) #Sort it in a descending order. Maximum first.
    print "Because u watched the movie : " + max_movie_name
    recommends = list(corr.index)
    recommends.pop(0)
else: #IF USER DOES NOT LIKE ANY MOVIE, RECOMMEND 5 HIGHEST IMDB RATED MOVIES.
    recommends = list(movies_data.sort_values("Rating",ascending=False).head(5)['Name']) #Get highest IMDB rated movies
for movie in recommends:
    print movie

Because u watched the movie : Miss Bala
It: Chapter Two
Toy Story 4
The Girl in the Spider's Web
Close
The Blacklist


# User - User

In [45]:
movie_user_rating=movie_data.pivot_table(index= 'Name',columns='UserID',values='Rated',aggfunc='first').astype(float)#UtlityMatrix
usern_rating = movie_user_rating[1000].astype(float) #Get ratings of new user(index = 1000)
user_alike=movie_user_rating.corrwith(usern_rating,method='pearson')#find user with similar ratings(pearson Similarity)
corr_user = pd.DataFrame(user_alike, columns=['Correlation'])  
corr_user.dropna(inplace=True)  
corr_user = corr_user.sort_values('Correlation',ascending=False).head(10)#Descending order to get maximum ratings first
similar_user = list(corr_user.index)[1] #Most similar user
similar_user_rating = movie_user_rating[similar_user] #Get Ratings of the most similar user
usern_rating.dropna(inplace=True)
similar_user_rating.dropna(inplace=True)
similar_user_rating = similar_user_rating.sort_values(ascending=False).head(6)#Highest rated movies of similar user
user_data = user_rating[user_rating.UserID == (1000)]
user_full = (user_data.merge(movies_data, how = 'left', left_on = 'MovieID', right_on = 'MovieID')
             .sort_values(['Rated'], ascending=False)) #find movies watched by new user
movies_watched = list(user_full['MovieID'])
recommended_movies = list(similar_user_rating.index)
for movie in recommended_movies:              #check if any recommended movie has already been rated by new user
    if movie in movies_watched:
        recommended_movies.pop(movie)         #remove from recommendations if already rated
recommended_movies.pop(0)
print "Recommended movies are : "
i=0
for a in recommended_movies:
    print a + "  " +str(similar_user_rating[i])
    i=i+1

Recommended movies are : 
Venom  5.0
How to Train Your Dragon 2  5.0
Unbreakable  5.0
Archer  5.0
Avatar: The Last Airbender  5.0


## Matrix Decomposition


In [59]:
matrix_df = user_rating.pivot_table(index='UserID',columns ='MovieID',values='Rated',aggfunc='first').astype(float).fillna(0)  #utility matrix
R = matrix_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)        #Demeaning the data
matrix_de_mean = R - user_ratings_mean.reshape(-1, 1)
U, sigma, Vt = svds(matrix_de_mean, k = 50)    #Singular Value Decomposition 
sigma = np.diag(sigma)                         #Converting into a diagonal matrix
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1) #Getting original data
prediction_df = pd.DataFrame(all_user_predicted_ratings, columns = user_movie_rating.columns) #predictions DF
def recommend_movies(predictions_df, userID, movies_df, users_ratings_df):
    user_number = userID-1  #Get User Index
    sorted_predictions = predictions_df.iloc[user_number].sort_values(ascending=False)
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
                 merge(pd.DataFrame(sorted_predictions).reset_index(), how = 'left').
                 rename(columns = {user_number: 'Predictions'}).
                 sort_values('Predictions', ascending = False).iloc[:5, :-1])
    return recommendations
predictions = recommend_movies(prediction_df, 1000, movies_data,user_rating)
recommended_movies = list(predictions['Name'].head(5))
recommended_movies

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


['American Horror Story',
 'Alita: Battle Angel',
 'Can You Ever Forgive Me?',
 'IO',
 'Frozen']